# Example usage

Here, we demonstrate how to use `fasterrisk` to generate sparse risk scoring systems:

## Imports

In [1]:
from fasterrisk.fasterrisk import RiskScoreOptimizer, RiskScoreClassifier
import urllib.request
import os.path

import numpy as np
import time

## Download Sample Data

In [2]:
sample_data_file_path = "../tests/train_test_data.npz"

if not os.path.isfile(sample_data_file_path):
    sample_data_link = "https://drive.google.com/file/d/1CK78YUUzDqx3dnPQ637hzrCFejU36eTU/view?usp=sharing"
    urllib.request.urlretrieve(sample_data_link, sample_data_file_path)

## Read Data

In [3]:
train_test_data = np.load("../tests/train_test_data.npz", allow_pickle=True)

X_train, y_train, X_test, y_test = train_test_data["X_train"], train_test_data["y_train"], train_test_data["X_test"], train_test_data["y_test"] 
y_train = y_train.reshape(-1)
y_test = y_test.reshape(-1)

## Create RiskScoreOptimizer and Perform Optimization

In [4]:
sparsity = 5
parent_size = 10

RiskScoreOptimizer_m = RiskScoreOptimizer(X = X_train, y = y_train, k = sparsity, parent_size = parent_size)

In [5]:
start_time = time.time()
RiskScoreOptimizer_m.optimize()
print("Optimization takes {:.2f} seconds.".format(time.time() - start_time))

Optimization takes 15.34 seconds.


## Get Risk Score Models

In [6]:
multipliers, sparse_diverse_set_integer = RiskScoreOptimizer_m.get_models()
print("We generate {} risk score models from the sparse diverse pool".format(len(multipliers)))

We generate 50 risk score models from the sparse diverse pool


### Access the first risk score model

In [7]:
model_index = 0 # first model
multiplier = multipliers[0]
intercept, coefficients = sparse_diverse_set_integer[0, model_index], sparse_diverse_set_integer[1:, model_index]

### Use the first risk score model to do prediction

In [8]:
RiskScoreClassifier_m = RiskScoreClassifier(multiplier, intercept, coefficients)

In [9]:
y_test_pred = RiskScoreClassifier_m.predict(X_test[:, 1:])
print("y_test are predicted to be {}".format(y_test_pred))

y_test are predicted to be [-1 -1 -1 ... -1 -1 -1]


In [10]:
y_test_pred_prob = RiskScoreClassifier_m.predict_prob(X_test[:, 1:])
print("The risk probabilities of having y_test to be +1 are {}".format(y_test_pred_prob))

The risk probabilities of having y_test to be +1 are [0.13308868 0.34872682 0.34872682 ... 0.04216029 0.34872682 0.04216029]
